# XML, HTML, and Web Scraping

JSON and XML are two different ways to represent hierarchical data. Which one is better? There are lots of articles online which discuss similarities and differences between JSON and XML and their advantages and disadvantages. Both formats are still in current usage, so it is good to be familiar with both. However, JSON is more common, so we'll focus on working with JSON representations of hierarchical data.

The reading covered an example of using Beautiful Soup to parse XML. Rather than doing another example XML now, we'll skip straight to scraping HTML from a webpage. Both HTML and XML can be parsed in a similar way with Beautiful Soup.

In [ ]:
import pandas as pd

## Scraping an HTML table with Beautiful Soup

Open the URL https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population and scroll down until you see a table of the cities in the U.S. with population over 100,000 (as of Jul 1, 2022). We'll use Beautiful Soup to scrape information from this table.

Read in the HTML from the ULR using the `requests` library.

In [ ]:
import requests
URL = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
response = requests.get(URL, headers=HEADERS)

Use Beautiful Soup to parse this string into a tree called `soup`

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

To find an HTML tag corresponding to a specific element on a webpage, right-click on it and choose "Inspect element". Go to the cities table Wikipedia page and do this now.

You should find that the cities table on the Wikipedia page corresponds to the element

```
<table class="wikitable sortable jquery-tablesorter" style="text-align:center">
```

There are many `<table>` tags on the page.

In [ ]:
len(soup.find_all("table"))

10

We can use attributes like `class=` and `style=` to narrow down the list.

In [ ]:
len(soup.find_all("table", {"class": "wikitable"}))

7

At this point, you can manually inspect the tables on the webpage to find that the one we want is the first one (see `[0]` below). We'll store this as `table`.

In [ ]:
table = soup.find_all("table",
                      attrs={
                          "class": "wikitable"}
                      )[0]


**Now you will write code to scrape the information in `table` to create a Pandas data frame with one row for each city and columns for: city, state, population (2022 estimate), and 2020 land area (sq mi).** Refer to the Notes/suggestions below as you write your code. A few Hints are provided further down, but try coding first before looking at the hints.

Notes/suggestions:

- Use as a guide the code from the reading that produced the data frame of Statistics faculty
- Inspect the page source as you write your code
- You will need to write a loop to get the information for all cities, but you might want to try just scraping the info for New York first
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `.get_text(strip = True)` instead of `.text`
- Don't forget to convert to a Pandas Data Frame; it should have 333 rows and 4 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it --- e.g., what is the population density for all cities in CA? --- then you would need to clean the data first (to clean strings and convert to quantitative). (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)

In [ ]:
# re run stuff from earlier
import pandas as pd
import requests
from bs4 import BeautifulSoup

URL = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
response = requests.get(URL, headers=HEADERS)
soup = BeautifulSoup(response.content, "html.parser")
table = soup.find("table", {"class": "wikitable"})


In [ ]:
# ran into a a bug and evtuatly foudn oiut the first table is actualy the key
all_tables = soup.find_all("table", {"class": "wikitable"})
table = all_tables[1]

# start list
rows = []

# accidnelty got the header so added [1:]
for city_row in table.find_all("tr")[1:]:

    # used the hint from bellow to know to use td
    cells = city_row.find_all("td")

    # have to skip 2020 censue and % cange to get to square miiles
    if len(cells) >= 5:

        # The rank column is represented by <th> tags, rather than <td> tags.
        # So within a row, the first (that is, [0]) <td> tag corresponds to the city name.
        city = cells[0].get_text(strip=True)
        state = cells[1].get_text(strip=True)
        population = cells[2].get_text(strip=True)
        land_area = cells[4].get_text(strip=True)

        # add to the list
        rows.append({
            "City": city,
            "State": state,
            "Population": population,
            "sq mi": land_area
        })

# make into pandas df
df_cities = pd.DataFrame(rows)

# check data is the right size
print(df_cities.shape)
print(df_cities.head(10))

(346, 4)
              City State Population   sq mi
0      New York[c]    NY  8,478,072  −3.70%
1      Los Angeles    CA  3,878,704  −0.51%
2          Chicago    IL  2,721,308  −0.91%
3          Houston    TX  2,390,125  +3.71%
4          Phoenix    AZ  1,673,164  +4.04%
5  Philadelphia[d]    PA  1,573,916  −1.86%
6      San Antonio    TX  1,526,656  +6.41%
7        San Diego    CA  1,404,452  +1.26%
8           Dallas    TX  1,326,087  +1.66%
9  Jacksonville[e]    FL  1,009,833  +6.34%


Hints:

- Each city is a row in the table; find all the `<tr>` tags to find all the cities
- Look for the `<td>` tag to see table entries within a row
- The rank column is represented by `<th>` tags, rather than `<td>` tags. So within a row, the first (that is, `[0]`) `<td>` tag corresponds to the city name.

## Scraping information that is NOT in a `<table>` with Beautiful Soup

The Cal Poly course catalog http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory contains a list of courses offered by the Statistics department. **You will scrape this website to obtain a Pandas data frame with one row for each DATA or STAT course and two columns: course name and number (e.g, DATA 301. Introduction to Data Science) and term typically offered (e.g., Term Typically Offered: F, W, SP).**

Note: Pandas `read_html` is not help here since the courses are not stored in a `<table>.`

In [ ]:
pd.read_html("http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory")

[                                        Program name   Program type
 0                              Actuarial Preparation          Minor
 1  Cross Disciplinary Studies Minor in Bioinforma...          Minor
 2   Cross Disciplinary Studies Minor in Data Science          Minor
 3                                         Statistics  BS, MS, Minor]


Notes/suggestions:


- Inspect the page source as you write your code
- The courses are not stored in a `<table>`. How are they stored?
- You will need to write a loop to get the information for all courses, but you might want to try just scraping the info for DATA 100 first
- What kind of tag is the course name stored in? What is the `class` of the tag?
- What kind of tag is the quarter(s) the course is offered stored in? What is the `class` of the tag? Is this the only tag of this type with the class? How will you get the one you want?
- You don't have to remove the number of units (e.g., 4 units) from the course name and number, but you can try it if you want
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `get_text(strip = True)` instead of `text`
- Don't forget to convert to a Pandas Data Frame; it should have 74 rows and 2 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it then you might need to clean the data first. (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)



In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

URL = "http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
response = requests.get(URL, headers=HEADERS)
soup = BeautifulSoup(response.content, "html.parser")


In [ ]:
# basicly the same thing as beffore but the tags are unique for each colom
rows = []


for course in soup.find_all("div", {"class": "courseblock"}):
    name_tag = course.find("p", {"class": "courseblocktitle"})
    course_name = name_tag.get_text(strip=True)

    term_tag = course.find("p", {"class": "noindent"})
    term_offered = term_tag.get_text(strip=True)

    rows.append({
        "Course": course_name,
        "Term Typically Offered": term_offered
    })

df_courses = pd.DataFrame(rows)


print(df_courses.shape)
print(df_courses.head(10))

(74, 2)
                                              Course  \
0           DATA 100. Data Science for All I.4 units   
1     DATA 301. Introduction to Data Science.4 units   
2  DATA 401. Data Science Process and Ethics.3 units   
3  DATA 402. Mathematical Foundations of Data Sci...   
4  DATA 403. Data Science Projects Laboratory.1 unit   
5        DATA 441. Bioinformatics Capstone I.2 units   
6       DATA 442. Bioinformatics Capstone II.2 units   
7          DATA 451. Data Science Capstone I.2 units   
8         DATA 452. Data Science Capstone II.2 units   
9              DATA 472. Data Science Seminar.1 unit   

             Term Typically Offered  
0  Term Typically Offered: F, W, SP  
1  Term Typically Offered: F, W, SP  
2         Term Typically Offered: F  
3         Term Typically Offered: F  
4         Term Typically Offered: F  
5         Term Typically Offered: W  
6        Term Typically Offered: SP  
7         Term Typically Offered: W  
8        Term Typically Offered: 

Hints:

- Each course is represented by a `<div>` with `class=courseblock`, so you can find all the courses with `soup.find_all("div", {"class": "courseblock"})`
- The course name is in a `<p>` tag with `class=courseblocktitle`, inside a `<strong>` tag. (Though I don't think we need to find the strong tag here.)
- The term typically offered is in `<p>` tag with `class=noindent`. However, there are several tags with this class; term typically offered is the first one.
- If you want to use Beautiful Soup to remove the course units (e.g., 4 units), find the `<span>` tag within the course name tag and `.extract()` this span tag